In [ ]:
!wget "https://raw.githubusercontent.com/NVIDIA/TensorRT/main/quickstart/IntroNotebooks/helper.py"

In [1]:
!nvidia-smi

Tue Jul  5 08:39:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.94       Driver Version: 470.94       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 55%   71C    P2   301W / 350W |  17715MiB / 24268MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:82:00.0 Off |                  N/A |
|  0%   

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import tensorflow as tf

print(tf.__version__)

2.6.0


In [3]:
from tensorflow.keras.applications import ResNet50

model_dir = 'data/resnet50_saved_model'
model = ResNet50(include_top=True, weights='imagenet')

2022-07-05 08:49:51.753674: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-05 08:49:52.418227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:82:00.0, compute capability: 8.6


In [6]:
model.save(model_dir) 

2022-05-31 23:41:12.601755: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: data/resnet50_saved_model/assets


/root/miniconda3/envs/trt_tf/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [4]:
import numpy as np

BATCH_SIZE = 32

dummy_input_batch = np.zeros((BATCH_SIZE, 224, 224, 3))

In [5]:

import tensorflow as tf
import tensorrt as trt

In [6]:
from tensorflow.python.compiler.tensorrt import trt_convert as tf_trt
from tensorflow.python.saved_model import tag_constants

In [7]:
PRECISION = "FP32" # Options are "FP32", "FP16", or "INT8"

In [18]:
from trt_help import ModelOptimizer # using the helper from <URL>

model_dir = 'data/resnet50_saved_model'

opt_model = ModelOptimizer(model_dir)

In [19]:
model_fp32 = opt_model.convert(model_dir+'_FP32', precision=PRECISION)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)


2022-07-05 10:20:18.628912: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-07-05 10:20:18.629129: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-05 10:20:18.631058: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:82:00.0, compute capability: 8.6
2022-07-05 10:20:18.681386: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 1202 nodes (878), 1857 edges (1533), time = 28.024ms.
  function_optimizer: function_optimizer did nothing. time = 1.378ms.

2022-07-05 10:20:21.381705: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-07-05 10:20:21.381836: I te

INFO:tensorflow:Assets written to: data/resnet50_saved_model_FP32/assets


In [20]:
model_fp32.predict(dummy_input_batch)

array([[1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       ...,
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04],
       [1.6970110e-04, 3.2997524e-04, 6.1348408e-05, ..., 1.4625140e-05,
        1.4453841e-04, 6.6052313e-04]], dtype=float32)

In [21]:
# Warm up - the first batch through a model generally takes longer
a = model.predict(dummy_input_batch)
b = model_fp32.predict(dummy_input_batch)
print(a.shape)
print(b.shape)
print(a)
print(b)

(32, 1000)
(32, 1000)
[[1.69701569e-04 3.29971721e-04 6.13481097e-05 ... 1.46249695e-05
  1.44537422e-04 6.60522783e-04]
 [1.69701569e-04 3.29971721e-04 6.13481097e-05 ... 1.46249695e-05
  1.44537422e-04 6.60522783e-04]
 [1.69701569e-04 3.29971721e-04 6.13481097e-05 ... 1.46249695e-05
  1.44537422e-04 6.60522783e-04]
 ...
 [1.69701569e-04 3.29971721e-04 6.13481097e-05 ... 1.46249695e-05
  1.44537422e-04 6.60522783e-04]
 [1.69701569e-04 3.29971721e-04 6.13481097e-05 ... 1.46249695e-05
  1.44537422e-04 6.60522783e-04]
 [1.69701569e-04 3.29971721e-04 6.13481097e-05 ... 1.46249695e-05
  1.44537422e-04 6.60522783e-04]]
[[1.6970110e-04 3.2997524e-04 6.1348408e-05 ... 1.4625140e-05
  1.4453841e-04 6.6052313e-04]
 [1.6970110e-04 3.2997524e-04 6.1348408e-05 ... 1.4625140e-05
  1.4453841e-04 6.6052313e-04]
 [1.6970110e-04 3.2997524e-04 6.1348408e-05 ... 1.4625140e-05
  1.4453841e-04 6.6052313e-04]
 ...
 [1.6970110e-04 3.2997524e-04 6.1348408e-05 ... 1.4625140e-05
  1.4453841e-04 6.6052313e-04]
 

In [22]:
%%timeit

model.predict_on_batch(dummy_input_batch)

52.6 ms ± 493 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%timeit

model_fp32.predict(dummy_input_batch)

31.3 ms ± 647 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
